# ESEF-Berichterstattung in Deutschland: Herausforderungen und Chancen

## Der Python-Code zum Replizieren der Tabellen 1–3

In [1]:
import pandas as pd

### Table 1

Tabelle 1 zeigt die ESEF-Konformitätsraten deut-scher kapitalmarktorientierter Unternehmen über den Beobachtungszeitraum von 2020 bis 2023. 

In [2]:
tb1_uregdw_s5 = pd.read_pickle("../data/tb1_uregdw_s5_20250616.p.gz")

In [3]:
def create_tb1_panel(df, byvar, ascending =True):
    # Group by the specified variable
    tb1_de_rows = df.groupby([byvar]).gvkey.nunique().reset_index().rename(columns={"gvkey": "gvkey_nunique"})
    
    # Create the different groupings
    tb1c_01 = df.groupby([byvar]).gvkey.count().reset_index()
    tb1c_02 = df.loc[df.sum_filing_score>=8].groupby([byvar]).gvkey.count().reset_index()
    tb1c_03 = df.loc[df.sum_filing_score>=16].groupby([byvar]).gvkey.count().reset_index()
    tb1c_04 = df.loc[df.sum_filing_score>=32768].groupby([byvar]).gvkey.count().reset_index()
    
    # Merge all the data
    result = pd.merge(tb1_de_rows, tb1c_01, on=byvar, how="left", suffixes=("", "")).fillna(0)
    result = pd.merge(result, tb1c_02, on=byvar, how="left", suffixes=("", "_esef")).fillna(0)
    result["anteil_esef"] = result["gvkey_esef"]/result["gvkey"]
    result["anteil_esef"] = result["anteil_esef"].apply(lambda x: round(x*100, 1))
    
    result = pd.merge(result, tb1c_03, on=byvar, how="left", suffixes=("", "_xbrl")).fillna(0)
    result["anteil_xbrl"] = result["gvkey_xbrl"]/result["gvkey"]
    result["anteil_xbrl"] = result["anteil_xbrl"].apply(lambda x: round(x*100, 1))
    
    result = pd.merge(result, tb1c_04, on=byvar, how="left", suffixes=("", "_complete")).fillna(0)
    result["anteil_complete"] = result["gvkey_complete"]/result["gvkey"]
    result["anteil_complete"] = result["anteil_complete"].apply(lambda x: round(x*100, 1))
    
    result.fillna(0, inplace=True)
    result.sort_values(by=byvar, ascending=ascending, inplace=True)
    result.to_excel(f"../results/tb1_by{byvar}.xlsx")
    return result


In [4]:
# Panel A
tb1_by_formyear = create_tb1_panel(tb1_uregdw_s5, "formyear")
tb1_by_formyear

,formyear,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,2020,596,596,354,59.4,330,55.4,317,53.2
1,2021,595,595,410,68.9,405,68.1,362,60.8
2,2022,572,572,397,69.4,392,68.5,355,62.1
3,2023,492,492,368,74.8,362,73.6,330,67.1


In [5]:
# table 1 2020--2023 ESEF Sample period
tb1_uregdw_s5["2020--2023"] = tb1_uregdw_s5.formyear.apply(lambda x: True if x>=2020 & x<=2024 else False)

byvar = "2020--2023"
tb1_esef = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=False)
tb1_esef

,2020--2023,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,True,610,2255,1529,67.8,1489,66.0,1364,60.5


In [6]:
byvar = "FSE_Label"
tb1_by_FSE_label = create_tb1_panel(tb1_uregdw_s5, byvar)
tb1_by_FSE_label

,FSE_Label,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,1. DAX40,42,163,157,96.3,157,96.3,155,95.1
1,2. Prime Standard,208,806,707,87.7,701,87.0,681,84.5
2,3. SME,82,299,220,73.6,213,71.2,186,62.2
3,4. Aktuell nicht als Aktie an FWB,282,987,445,45.1,418,42.4,342,34.7


In [7]:
tb1_uregdw_s5["DE_ISIN"] = tb1_uregdw_s5["isin"].str.startswith("DE")
tb1_uregdw_s5["DE_ISIN"] = tb1_uregdw_s5["DE_ISIN"].fillna(True)

byvar = "DE_ISIN"
tb1_de_isin = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=False)
tb1_de_isin

/var/folders/mz/hx4rhsms565c3zkfh7qx8xs00000gn/T/ipykernel_43243/187706699.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tb1_uregdw_s5["DE_ISIN"] = tb1_uregdw_s5["DE_ISIN"].fillna(True)


,DE_ISIN,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
1,True,590,2189,1492,68.2,1452,66.3,1330,60.8
0,False,20,66,37,56.1,37,56.1,34,51.5


In [8]:
byvar = "mcap_g4"
tb1_de_mcap_beg = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=False)
tb1_de_mcap_beg

,mcap_g4,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
3,MCap4,160,528,360,68.2,353,66.9,312,59.1
2,MCap3,159,476,359,75.4,357,75.0,342,71.8
1,MCap2,219,661,433,65.5,420,63.5,380,57.5
0,MCap1,186,590,377,63.9,359,60.8,330,55.9


In [9]:
byvar = "gsec_type"
tb1_de_mcap_beg = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=True)
tb1_de_mcap_beg

,gsec_type,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,10 Energie,7,26,13,50.0,13,50.0,12,46.2
1,15 Roh- und Grundstoffe,32,115,94,81.7,89,77.4,77,67.0
2,20 Industrie,136,519,398,76.7,395,76.1,373,71.9
3,25 Verbraucher Diskretionäre,90,329,223,67.8,218,66.3,196,59.6
4,30 Verbraucher Staples,25,92,44,47.8,44,47.8,44,47.8
5,35 Gesundheitswesen,61,222,145,65.3,135,60.8,124,55.9
6,40 Finanzen,53,206,160,77.7,152,73.8,126,61.2
7,45 Informationstechnologie,109,386,201,52.1,200,51.8,194,50.3
8,50 Telekommunikation,46,167,111,66.5,108,64.7,101,60.5
9,55 Energieversorgung,16,61,42,68.9,41,67.2,36,59.0
